<a href="https://colab.research.google.com/github/schesa/ai-memes/blob/master/ai_memes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Check GPU


In [0]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    # print(local_device_protos)
    return [x.physical_device_desc for x in local_device_protos if x.device_type == 'GPU']
get_available_gpus()    

['device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7']

# Mount Drive

In [0]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    print('Note: using Google CoLab')
    COLAB = True
except:
    print('Note: not using Google Colab')
    COLAB = False

if COLAB:
    root_path = "/content/drive/My Drive/licenta AC/Chesa/data"
else:
    root_path = "./data/captions"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Note: using Google CoLab


# Get Dataset
* **root_captioning** = "/content/drive/My Drive/licenta AC/Chesa/data"
* **lookup** -> dict key: Meme id, value list captions containing words & punctuations splitted by space
* delete punctuations so that words will be found in Glove
* remove non ascii captions
* lex -> set of unique words
* max_length -> Maximum length of a caption (in words)

# Prepare Dataset
* split into **train** and **test**
* train_images set, train_img list, string path '3001612175_53567ffb58.jpg'
* train_descriptions set, add start/stop get from lookup by train_image 
  ['2092870249_90e3f1855b': ['startseq a ... , ... a jack . endseq'],]
* all_train_captions - all merged in list

# Build Neural Networks
* build **InceptionV3**
* encode img features in **train.pkl**
* build vocab, cut words with least occurrences
* load **glove** embeddings
* build embedding matrix for vocab
* build **LSTM**

# Caption
* implement caption prediction beam search



In [0]:
import string

null_punct = str.maketrans('', '', string.punctuation)

def clean_data(list):
  for desc in list:
    desc = [word.lower() for word in desc.split()]
    desc = [w.translate(null_punct) for w in desc]
    desc = [word for word in desc if word.isalpha()]
    desc = [word for word in desc if len(word)>1]
    yield ' '.join(desc)

In [0]:
import os
from pathlib import Path
import json
from tqdm import tqdm
from functools import reduce

isascii_word = lambda w: len(w) == len(w.encode())
isascii_list = lambda l: reduce(lambda rez, word: rez & True if isascii_word(word) else False & rez, l, True)
dirname = os.path.join(root_path, 'ImgFlip500K_Dataset', 'memes')
lookup = dict()

def print_iterator(it):
    for x in it:
        print(x, end='\n')
    print('')  # for new line
# i = 0
for filename in tqdm(os.listdir(dirname)): # foreach json file in memes
    # i += 1
    # if i < 7:
    #   continue
    meme_name = Path(filename).stem # remove file extension
    with open(os.path.join(dirname, filename)) as json_file:
      memes = json.load(json_file)
      # lookup[meme_name] = list(map(lambda meme: ' | '.join(meme['boxes']), memes))
      lookup[meme_name] = []
      for meme in memes:
        words = ' | '.join(clean_data(meme['boxes']))
        if isascii_list(words):
          lookup[meme_name].append(words)
    # if i>0:
    #   break # TODO REMOVE

print(f'Memes loaded: {len(lookup)}') # 99 memes, in the latest dataset 100
print(f'Meme example: {lookup["Mocking-Spongebob"][0]}') # when you off the dope | and you think you a bird

100%|██████████| 99/99 [01:04<00:00,  1.54it/s]

Memes loaded: 99
Meme example: when you off the dope | and you think you bird


In [0]:
max_length = 0
lex = set() # set of unique words
# for key in lookup:
#   [lex.update(d.split()) for d in lookup[key]]
#   big_word = reduce(lambda l, rez: len(max(l.split(), key=lambda x: len(x))) if len(max(l.split(), key=lambda x: len(x))) > len(rez) else rez, lookup[key])
#   if(len(big_word)> max_length):
#     max_length = len(big_word)
#     max_word = big_word
#   break
for desc in lookup:
  for word in desc.split():
    lex.add(word)
    if max_length < len(word):
      max_length=max(max_length,len(word))
      max_word = word

print(f'Unique words: {len(lex)}')
print(f'Biggest word: {max_length} chars')
max_word


Unique words: 99
Biggest word: 46 chars


'Bernie-I-Am-Once-Again-Asking-For-Your-Support'

In [0]:
START = "startseq"
STOP = "endseq"
# same as lookup but captions wrapped in Start/Stop
train_descriptions = {k:v for k,v in lookup.items()}
for n,v in train_descriptions.items(): 
  for d in range(len(v)):
    v[d] = f'{START} {v[d]} {STOP}'
max_length += 2 # update maximum caption length
print(f'Wrapped captions in Start/Stop')

Wrapped captions in Start/Stop


In [0]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
import tensorflow.keras.applications.inception_v3
from tensorflow.keras.models import Model

encode_model = InceptionV3(weights='imagenet')
encode_model = Model(encode_model.input, encode_model.layers[-2].output)
WIDTH = 299
HEIGHT = 299
OUTPUT_DIM = 2048
preprocess_input = tensorflow.keras.applications.inception_v3.preprocess_input

print('InceptionV3 loaded!')

96116736/96112376 [==============================] - 2s 0us/step
InceptionV3 loaded!


In [0]:
from PIL import Image
import tensorflow.keras.preprocessing.image
import numpy as np

def encodeImage(img):
  # Resize all images to a standard size (specified bythe image encoding network)
  img = img.resize((WIDTH, HEIGHT), Image.ANTIALIAS)
  # Convert a PIL image to a numpy array
  x = tensorflow.keras.preprocessing.image.img_to_array(img)
  # Expand to 2D array
  x = np.expand_dims(x, axis=0)
  # Perform any preprocessing needed by InceptionV3 or others
  x = preprocess_input(x)
  # Call InceptionV3 (or other) to extract the smaller feature set for the image.
  x = encode_model.predict(x) # Get the encoding vector for the image
  # Shape to correct form to be accepted by LSTM captioning network.
  x = np.reshape(x, OUTPUT_DIM )
  return x

In [0]:
# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return f"{h}:{m:>02}:{s:>05.2f}"

In [0]:
import pickle
from time import time

train_path = os.path.join(root_path,'ImgFlip500K_Dataset',"data",f'train{OUTPUT_DIM}.pkl')
if not os.path.exists(train_path):
  start = time()
  encoding_train = {}
  train_images_path = os.path.join(root_path,'ImgFlip500K_Dataset','templates','img') 
  for image_path in tqdm(os.listdir(train_images_path)):
    # print(image_path)
    img = tensorflow.keras.preprocessing.image.load_img(os.path.join(train_images_path,image_path), target_size=(HEIGHT, WIDTH))
    encoding_train[image_path] = encodeImage(img)
  with open(train_path, "wb") as fp:
    pickle.dump(encoding_train, fp)
  print(f"\nGenerating training set took: {hms_string(time()-start)}")
else:
  with open(train_path, "rb") as fp:
    encoding_train = pickle.load(fp)
print('Loaded')

Loaded


In [0]:
all_train_captions = []
for key, val in train_descriptions.items():
    for cap in val:
        all_train_captions.append(cap)
# print(all_train_captions)

Words that do not occur very often can be misleading to neural network training.  It is better to simply remove such words.  Here we remove any words that occur less than 10 times.  We display what the total vocabulary shrunk to.

In [0]:
word_count_threshold = 1
word_counts = {}
nsents = 0 # number of sentences
for sent in all_train_captions:
    nsents += 1
    for w in sent.split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1

vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]
print('preprocessed words %d ==> %d' % (len(word_counts), len(vocab)))

preprocessed words 127547 ==> 127547


Next we build two lookup tables for this vocabulary. One idxtoword convers index numbers to actual words to index values.  The wordtoidx lookup table performs the opposit.

In [0]:
idxtoword = {}
wordtoidx = {}

ix = 1
for w in vocab:
    wordtoidx[w] = ix
    idxtoword[ix] = w
    ix += 1
    
vocab_size = len(idxtoword) + 1 
vocab_size

127548

 # Loading Glove Embeddings

In [0]:
glove_dir = os.path.join(root_path,'glove.6B')
embeddings_index = {} 
f = open(os.path.join(glove_dir, 'glove.6B.200d.txt'), encoding="utf-8")

for line in tqdm(f):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

f.close()
print(f'Found {len(embeddings_index)} word vectors.')

400000it [00:23, 16855.93it/s]

Found 400000 word vectors.


An embedding matrix is built from Glove.  This will be directly copied to the weight matrix of the neural network.

In [0]:
embedding_dim = 200

# Get 200-dim dense vector for each of the 10000 words in out vocabulary
embedding_matrix = np.zeros((vocab_size, embedding_dim))
not_found = []
for word, i in wordtoidx.items():
    #if i < max_words:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in the embedding index will be all zeros
        embedding_matrix[i] = embedding_vector
    else: 
      not_found.append(word)
print(f'Not founded words: {len(not_found)} from total of {len(wordtoidx)}')


Not founded words: 67437 from total of 127547


Clean captions that contain not founded words

In [0]:
eliminated = 0
def hasNotFoundWord(desc):
  global eliminated
  stripped_desc = desc.split()[1:-1]
  if bool(set(not_found) & set(stripped_desc)):
    eliminated += 1
    return True
  else:
    return False

train_desc_path = os.path.join(root_path,'ImgFlip500K_Dataset',"data",f'{train_descriptions}.pkl')
if not os.path.exists(train_desc_path):
  for meme in tqdm(train_descriptions):
    train_descriptions[meme] = [desc for desc in train_descriptions[meme] if hasNotFoundWord(desc) == False ]
    # train_descriptions[meme] = [desc for desc in train_descriptions[meme] if all(unknown not in desc.split()[1:-1] for unknown in not_found)]
  # with open(train_desc_path, "wb") as fp:
  #   pickle.dump(train_descriptions, fp)
  print(f"\nGenerated Training Descriptions")
else:
  with open(train_desc_path, "rb") as fp:
    train_descriptions = pickle.load(fp)
  print(f"\n Loaded")

100%|██████████| 99/99 [29:00<00:00, 17.58s/it]


Generated Training Descriptions


In [0]:
# print(f"\nGenerated Training Descriptions")
all_train_captions = []
for key, val in train_descriptions.items():
    for cap in val:
        all_train_captions.append(cap)
# print(f'Removed captions: {eliminated} from total of {len(all_train_captions)}')

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import Input, layers
from tensorflow.keras import optimizers
from tensorflow.keras.layers import add

inputs1 = Input(shape=(OUTPUT_DIM,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)
inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)
decoder1 = add([fe2,se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)
caption_model = Model(inputs=[inputs1,inputs2], outputs=outputs)

In [0]:
# caption_model.summary()

In [0]:
caption_model.layers[2].set_weights([embedding_matrix])
caption_model.layers[2].trainable = False

caption_model.compile(loss='categorical_crossentropy', optimizer='adam')

# Train Nework

In [0]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

def data_generator(descriptions, photos, wordtoidx, max_length, num_photos_per_batch, num_descriptions_per_batch):
  # x1 - Training data for photos
  # x2 - The caption that goes with each photo
  # y - The predicted rest of the caption
  x1, x2, y = [], [], []
  n=0
  while True:
    for key, desc_list in descriptions.items():
      n+=1
      # print(key+'.jpg')
      photo = photos[key+'.jpg']
      d=0
      # Each photo has 5 descriptions
      for desc in desc_list:
        d+=1
        if d==num_descriptions_per_batch:
          yield ([np.array(x1), np.array(x2)], np.array(y))
          x1, x2, y = [], [], []
          d=0
        # Convert each word into a list of sequences.
        seq = [wordtoidx[word] for word in desc.split(' ') if word in wordtoidx]
        # Generate a training case for every possible sequence and outcome
        for i in range(1, len(seq)):
          in_seq, out_seq = seq[:i], seq[i]
          in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
          out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
          x1.append(photo)
          x2.append(in_seq)
          y.append(out_seq)
      # if n==num_photos_per_batch:
      #   yield ([np.array(x1), np.array(x2)], np.array(y))
      #   x1, x2, y = [], [], []
      #   n=0
    # break  # TODO REMOVE

In [0]:
# EPOCHS = 1
# model_path = os.path.join(root_path,'ImgFlip500K_Dataset',"data",f'caption-model.hdf5')
# if not os.path.exists(model_path):
#   for i in tqdm(range(EPOCHS*2)):
#       generator = data_generator(train_descriptions, encoding_train, wordtoidx, max_length, number_pics_per_bath)
#       caption_model.fit(generator, epochs=1, steps_per_epoch=steps, verbose=1)

#   caption_model.optimizer.lr = 1e-4
#   number_pics_per_bath = 6
#   steps = len(train_descriptions)//number_pics_per_bath

#   for i in range(EPOCHS):
#       generator = data_generator(train_descriptions, encoding_train, wordtoidx, max_length, number_pics_per_bath)
#       caption_model.fit(generator, epochs=1, steps_per_epoch=steps, verbose=1)  
#   caption_model.save_weights(model_path)
#   print(f"\Training took: {hms_string(time()-start)}")
# else:
#   caption_model.load_weights(model_path)

In [0]:
number_pics_per_bath = 1

EPOCHS = 10
#steps = 1 # TODO REMOVE
start = time()
num_descriptions_per_batch=30
steps = (len(all_train_captions))//num_descriptions_per_batch-1
print(f'steps: {steps}')
model_path = os.path.join(root_path,'ImgFlip500K_Dataset',"data",'caption-model.hdf5')
if not os.path.exists(model_path):
  # for i in tqdm(range(EPOCHS)):
  #   print(f'i: {i}')
  generator = data_generator(train_descriptions, encoding_train, wordtoidx, max_length, number_pics_per_bath, num_descriptions_per_batch)
  print(f'\n Fitting Steps1 {steps}\n')
  caption_model.fit(generator, epochs=EPOCHS, steps_per_epoch=steps, verbose=1)
  print(f'\n Done fitting\n')

  caption_model.optimizer.lr = 1e-4
  # number_pics_per_bath = 6
  # steps = len(train_descriptions)//number_pics_per_bath
  #steps=1

        # num_descriptions_per_batch=50
        # EPOCHS = 30
        # steps = (len(all_train_captions))//num_descriptions_per_batch-1
        # generator = data_generator(train_descriptions, encoding_train, wordtoidx, max_length, number_pics_per_bath, num_descriptions_per_batch)
        # print(f'\n Fitting Steps1 {steps}\n')
        # caption_model.fit(generator, epochs=EPOCHS, steps_per_epoch=steps, verbose=2)

  # steps = len(all_train_captions)//num_descriptions_per_batch
  # for i in tqdm(range(EPOCHS)):
  #     generator = data_generator(train_descriptions, encoding_train, wordtoidx, max_length, number_pics_per_bath, num_descriptions_per_batch)
  #     print(f'\n Steps2 {steps}\n')
  #     caption_model.fit(generator, epochs=1, steps_per_epoch=steps, verbose=1)  
  caption_model.save_weights(model_path)
  print(f"\Training took: {hms_string(time()-start)}")
else:
  caption_model.load_weights(model_path)
  print('Model loaded')

steps: 15070

 Fitting Steps1 15070

Epoch 1/10
15070/15070 [==============================] - 4025s 267ms/step - loss: 5.3889
Epoch 2/10
 1197/15070 [=>............................] - ETA: 1:02:16 - loss: 5.0784

In [0]:
def generateCaption(photo):
    in_text = START
    for i in range(max_length):
        sequence = [wordtoidx[w] for w in in_text.split() if w in wordtoidx]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = caption_model.predict([photo,sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = idxtoword[yhat]
        in_text += ' ' + word
        print(word)
        if word == STOP:
            break
    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final

In [0]:
  import matplotlib.pyplot as plt

  #img = 'Mocking-Spongebob.jpg'
  #img = 'I-Should-Buy-A-Boat-Cat.jpg'
  for img in train_descriptions.keys():
    image = encoding_train[f'{img}.jpg'].reshape((1,OUTPUT_DIM))
    x = plt.imread(os.path.join(root_path,'ImgFlip500K_Dataset','templates','img', f'{img}.jpg'))
    plt.imshow(x)
    plt.show()
    print(image)
    print("Caption:",generateCaption(image))
    print("_____________________________________")

In [0]:
len(all_train_captions)

In [0]:
def beam_search_predictions(image, beam_index = 3):
    start = [wordtoidx[START]]
    
    # start_word[0][0] = index of the starting word
    # start_word[0][1] = probability of the word predicted
    start_word = [[start, 0.0]]
    
    while len(start_word[0][0]) < max_length:
        temp = []
        for s in start_word:
            par_caps = pad_sequences([s[0]], maxlen=max_length, padding='post')
            e = encoding_train[image]
            preds = caption_model.predict([np.array([e]), np.array(par_caps)])
            
            # Getting the top <beam_index>(n) predictions
            word_preds = np.argsort(preds[0])[-beam_index:]
            
            # creating a new list so as to put them via the model again
            for w in word_preds:
                # print('w')
                # print(idxtoword[w])
                next_cap, prob = s[0][:], s[1]
                next_cap.append(w)
                prob += preds[0][w]
                temp.append([next_cap, prob])
                    
        start_word = temp
        print(temp)
        # Sorting according to the probabilities
        start_word = sorted(start_word, reverse=False, key=lambda l: l[1])
        # Getting the top words
        start_word = start_word[-beam_index:]
    print('start_word')
    for caption in start_word[-1]:
      print([idxtoword[i] for i in caption])
    print(start_word)
    start_word = start_word[-1][0]
    intermediate_caption = [idxtoword[i] for i in start_word]
    print(intermediate_caption)
    final_caption = []
    
    for i in intermediate_caption:
        if i != STOP:
            final_caption.append(i)
        else:
            break
    
    final_caption = ' '.join(final_caption[1:])
    return final_caption

In [0]:
# for img in train_descriptions.keys():
#   img =  f'{img}.jpg'
#   print(beam_search_predictions(img, 7))
#   x = plt.imread(os.path.join(root_path,'ImgFlip500K_Dataset','templates','img',img))
#   plt.imshow(x)
#   plt.show()
#   print(image)

In [0]:
    img = 'I-Should-Buy-A-Boat-Cat.jpg'
    photo = encoding_train[img].reshape((1,OUTPUT_DIM))
    in_text = START
    # for i in range(max_length):
    sequence = [wordtoidx[w] for w in in_text.split() if w in wordtoidx]
    sequence = pad_sequences([sequence], maxlen=max_length)
    yhat = caption_model.predict([photo,sequence], verbose=0)
    idxtoword[0]='0'
    # np.where(yhat[0] == yhat[0][2])[0][0]    
    probab = [[np.where(yhat[0] == p)[0][0], p] for p in list(yhat[0])]
    probab = [ [idxtoword[x[0]],x[1]] for x in probab if x[1]>0.01]
    probab

In [0]:
def beam_search(img, beam_index = 3):
  start = [wordtoidx[START]]
  start_word = [[start, 0.0]]

  while len(start_word[0][0]) < max_length:

    temp = []
    for s in start_word:

      par_caps = pad_sequences([s[0]], maxlen=max_length, padding='post')
      e = encoding_train[img]
      preds = caption_model.predict([np.array([e]), np.array(par_caps)])

      word_preds = np.argsort(preds[0])[-beam_index:]
      # print(word_preds)
      # creating a new list so as to put them via the model again
      for w in word_preds:
          # print(w)
          # print(idxtoword(w[0]))
          next_cap, prob = s[0][:], s[1]
          # print(next_cap)
          # print(idxtoword[next_cap[0]])
          next_cap.append(w)
          prob += preds[0][w]
          temp.append([next_cap, prob])
                              
          start_word = temp
          # print(temp)
          # Sorting according to the probabilities
          start_word = sorted(start_word, reverse=False, key=lambda l: l[1])
          # Getting the top words
          start_word = start_word[-beam_index:]
          
  # print('start_word')
  # print(start_word)
  l = []
  for rez in start_word:
     l.append([[idxtoword[i] for i in rez[0]],rez[1]])
  return l

    # print([[idxtoword[i] for i in caption[0]],caption[1]])
  # start_word = start_word[-1][0]
  # intermediate_caption = [idxtoword[i] for i in start_word]
  # print(intermediate_caption)
  # final_caption = []

  # for i in intermediate_caption:
  #     if i != STOP:
  #         final_caption.append(i)
  #     else:
  #         break

  # final_caption = ' '.join(final_caption[1:])
  # print(final_caption)

In [0]:
for img in train_descriptions.keys():
  img =  f'{img}.jpg'
  x = plt.imread(os.path.join(root_path,'ImgFlip500K_Dataset','templates','img',img))
  plt.imshow(x)
  plt.show()
  print(img)
  for caption in beam_search(img, 30):
    print(caption[0])


In [0]:
def diff_beam_search(img, beam_index = 3):
  start = [wordtoidx[START]]
  start_word = [[start, 0.0]]
  first = True
  while len(start_word[0][0]) < max_length:

    temp = []
    for s in start_word:

      par_caps = pad_sequences([s[0]], maxlen=max_length, padding='post')
      preds = caption_model.predict([np.array([img]), np.array(par_caps)])

      # if first == True:
      #   np.random.shuffle(preds)
      #   word_preds = preds[0][-beam_index:]
      # else:
      word_preds = np.argsort(preds[0])[-beam_index:]
      # print(word_preds)
      # creating a new list so as to put them via the model again
      for w in word_preds:
          # print(w)
          # print(idxtoword(w[0]))
          next_cap, prob = s[0][:], s[1]
          # print(next_cap)
          # print(idxtoword[next_cap[0]])
          next_cap.append(w)
          prob += preds[0][w]
          temp.append([next_cap, prob])
                              
          start_word = temp
          # print(temp)
          # Sorting according to the probabilities
          start_word = sorted(start_word, reverse=False, key=lambda l: l[1])
          # Getting the top words
          start_word = start_word[-beam_index:]
  # print('start_word')
  # print(start_word)
  l = []
  for rez in start_word:
     l.append([[idxtoword[i] for i in rez[0]],rez[1]])
  return l

In [0]:
for img in train_descriptions.keys():
  img = tensorflow.keras.preprocessing.image.load_img(os.path.join(root_path,'custom_images','WIN_20190821_08_34_54_Pro.jpg'), target_size=(HEIGHT, WIDTH))
  x = plt.imread(os.path.join(root_path,'custom_images','WIN_20190821_08_34_54_Pro.jpg'))
  plt.imshow(x)
  plt.show()
  print(img)
  for caption in diff_beam_search(encodeImage(img), 6):
    print(caption[0])